# Create a file from two datasets and add headers. Then use for "item base category" algorithm. 

We'll start by loading up the Shopping Rates dataset. Using Pandas, we can very quickly load the rows of the u.data and u.item files that we care about, and merge them together so we can work with items names instead of ID's. (In a real production job, you'd stick with ID's and worry about the names at the display layer to make things more efficient. But this lets us understand what's going on better for now.)

In [1]:
import pandas as pd

r_cols = ['product_id','user_id' , 'rating']
ratings = pd.read_csv('C:\\Users\\Diego Alves\\Desktop\\Data_sets\\test_jupyter.csv', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['product_id', 'title']
products = pd.read_csv('C:\\Users\\Diego Alves\Desktop\\Data_sets\\test_jupyter_2.csv', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(products, ratings)

In [2]:
ratings.head()

,product_id,title,user_id,rating
0,1,WHITE HANGING HEART T-LIGHT HOLDER,102,3
1,2,WHITE METAL LANTERN,109,2
2,3,CREAM CUPID HEARTS COAT HANGER,109,5
3,4,KNITTED UNION FLAG HOT WATER BOTTLE,104,3
4,5,RED WOOLLY HOTTIE WHITE HEART.,103,5


Now the amazing pivot_table function on a DataFrame will construct a user / movie rating matrix. Note how NaN indicates missing data - movies that specific users didn't rate.

In [3]:
productsRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
productsRatings.head()

title,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
user_id,,,,,,,,,,,,,,,,,,,,,
101,3.500000,3.222222,3.000000,2.200000,NaN,3.666667,2.846154,3.083333,3.138889,2.850000,...,NaN,4.0,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN
102,3.333333,2.866667,3.157895,2.642857,NaN,NaN,2.809524,3.357143,3.100000,2.647059,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,3.250000,3.727273,2.900000,2.625000,NaN,2.000000,2.900000,3.000000,3.346154,3.000000,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
104,3.500000,3.266667,3.071429,3.250000,NaN,2.000000,2.764706,3.200000,2.692308,2.909091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,3.000000,2.100000,2.782609,2.200000,4.0,2.000000,3.700000,3.428571,2.533333,3.384615,...,4.5,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


Let's extract a Series of users who rated "DOLLY GIRL BEAKER":

In [4]:
WHITE_METAL_LANTERN_Ratings = productsRatings['WHITE METAL LANTERN']
WHITE_METAL_LANTERN_Ratings.head()

user_id
101    3.285714
102    3.224490
103    3.189189
104    3.088235
105    2.576923
Name: WHITE METAL LANTERN, dtype: float64

Pandas' corrwith function makes it really easy to compute the pairwise correlation of WHITE METAL LANTERN vector of user rating with every other prduct! After that, we'll drop any results that have no data, and construct a new DataFrame of products and their correlation score (similarity) to WHITE METAL LANTERN:

In [5]:
similarProducts = productsRatings.corrwith(WHITE_METAL_LANTERN_Ratings)
similarProducts = similarProducts.dropna()
df = pd.DataFrame(similarProducts)
df.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,0
title,
4 PURPLE FLOCK DINNER CANDLES,0.211416
50'S CHRISTMAS GIFT BAG LARGE,0.435368
DOLLY GIRL BEAKER,0.257584
I LOVE LONDON MINI BACKPACK,-0.005706
NINE DRAWER OFFICE TIDY,0.131937
OVAL WALL MIRROR DIAMANTE,-0.562115
RED SPOT GIFT BAG LARGE,-0.084893
SET 2 TEA TOWELS I LOVE LONDON,0.530502
SPACEBOY BABY GIFT SET,-0.715176


(That warning is safe to ignore.) Let's sort the results by similarity score, and we should have the products most similar to WHITE METAL LANTERN! Except... we don't. These results make no sense at all! This is why it's important to know your data - clearly we missed something important.

In [6]:
similarProducts.sort_values(ascending=False)

title
RED STONE/CRYSTAL EARRINGS             1.0
DAIRY MAID  PUDDING BOWL               1.0
WRAP  VINTAGE DOILEY                   1.0
LEAVES MAGNETIC  SHOPPING LIST         1.0
VINTAGE PHOTO ALBUM PARIS DAYS         1.0
                                      ... 
SET 10 MINICARDS CUTE SNOWMAN 17071   -1.0
PADS TO MATCH ALL CUSHIONS            -1.0
PACK OF 12 DOILEY TISSUES             -1.0
ORANGE PENDANT TRIPLE SHELL NECKLAC   -1.0
wet/rusty                             -1.0
Length: 3850, dtype: float64

Our results are probably getting messed up by movies that have only been viewed by a handful of people who also happened to like WHITE METAL LANTERN. So we need to get rid of products that were only bought by a few people that are producing spurious results. Let's construct a new DataFrame that counts up how many ratings exist for each product, and also the average rating while we're at it - that could also come in handy later.

## Let's try to improve theses results:

Can you improve on these results? Perhaps a different method or min_periods value on the correlation computation would produce more interesting results.

Also, it looks like some movies similar to Gone with the Wind - which I hated - made it through to the final list of recommendations. Perhaps movies similar to ones the user rated poorly should actually be penalized, instead of just scaled down?

There are also probably some outliers in the user rating data set - some users may have rated a huge amount of movies and have a disporportionate effect on the results. Go back to earlier lectures to learn how to identify these outliers, and see if removing them improves things.

For an even bigger project: we're evaluating the result qualitatively here, but we could actually apply train/test and measure our ability to predict user ratings for movies they've already watched. Whether that's actually a measure of a "good" recommendation is debatable, though!

In [7]:
import numpy as np
productStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
productStats.head()

rating          
                                 size      mean
title                                          
 4 PURPLE FLOCK DINNER CANDLES     41  3.219512
 50'S CHRISTMAS GIFT BAG LARGE    130  3.069231
 DOLLY GIRL BEAKER                181  2.944751
 I LOVE LONDON MINI BACKPACK       88  2.647727
 I LOVE LONDON MINI RUCKSACK        1  4.000000

Let's get rid of any products rated by fewer than 100 people, and check the top-rated ones that are left:

In [8]:
popularProducts = productStats['rating']['size'] >= 100
productStats[popularProducts].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                      size      mean
title                                               
12 EGG HOUSE PAINTED WOOD              100  3.540000
SET 6 FOOTBALL CELEBRATION CANDLES     109  3.477064
HEARTS  STICKERS                       120  3.375000
MINI LADLE LOVE HEART RED              155  3.361290
RED REFECTORY CLOCK                    112  3.348214
CHRISTMAS METAL POSTCARD WITH BELLS    116  3.344828
LARGE CAKE TOWEL CHOCOLATE SPOTS       109  3.330275
MAGNETS PACK OF 4 VINTAGE LABELS       128  3.304688
COFFEE MUG DOG + BALL DESIGN           205  3.302439
DOORMAT WELCOME SUNRISE                123  3.300813
SWEETHEART WIRE WALL TIDY              113  3.292035
HOME GARLAND PAINTED ZINC              130  3.284615
TRAVEL CARD WALLET UNION JACK          235  3.272340
T-LIGHT HOLDER HANGING LOVE BIRD       103  3.271845
FILIGREE HEART BUTTERFLY WHITE         119  3.268908

100 might still be too low, but these results look pretty good as far as "well rated products that people have heard of." Let's join this data with our original set of similar products to HITE METAL LANTERN:

In [9]:
df = productStats[popularProducts].join(pd.DataFrame(similarProducts, columns=['similarity']))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [10]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
50'S CHRISTMAS GIFT BAG LARGE,130,3.069231,0.435368
DOLLY GIRL BEAKER,181,2.944751,0.257584
OVAL WALL MIRROR DIAMANTE,162,2.950617,-0.562115
RED SPOT GIFT BAG LARGE,105,3.200000,-0.084893
SET 2 TEA TOWELS I LOVE LONDON,282,2.865248,0.530502


And, sort these new results by similarity score. That's more like it!

In [11]:
df.sort_values(['similarity'], ascending=False)[:15]

,"(rating, size)","(rating, mean)",similarity
title,,,
WHITE METAL LANTERN,328,2.926829,1.000000
NATURAL SLATE RECTANGLE CHALKBOARD,427,3.032787,0.878113
MINI JIGSAW DINOSAUR,103,3.097087,0.874610
"KEY FOB , SHED",380,3.076316,0.844699
SET 12 KIDS COLOUR CHALK STICKS,376,2.888298,0.831866
SET OF 6 HEART CHOPSTICKS,159,2.880503,0.819564
SET/2 RED RETROSPOT TEA TOWELS,403,2.853598,0.810626
METAL SIGN CUPCAKE SINGLE HOOK,113,3.115044,0.806738
VINTAGE RED ENAMEL TRIM JUG,146,2.945205,0.806278


Ideally we'd also filter out the product we started from - of course WHITE METAL LANTERN is 100% similar to itself. But otherwise these results aren't bad.